In [1]:
import pandas as pd

In [5]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\THINKPAD\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [30]:
all_sheets = pd.read_excel("online_retail.xlsx", sheet_name=None)  # dict of DataFrames
data = pd.concat(all_sheets.values(), ignore_index=True)
data.shape


(1067371, 8)

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  object        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer ID  824364 non-null   float64       
 7   Country      1067371 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 65.1+ MB


In [32]:
data.isnull().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

In [33]:
data.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,1.067371e+06,1067371,1.067371e+06,824364.000000
mean,9.938898e+00,2011-01-02 21:13:55.394028544,4.649388e+00,15324.638504
min,-8.099500e+04,2009-12-01 07:45:00,-5.359436e+04,12346.000000
25%,1.000000e+00,2010-07-09 09:46:00,1.250000e+00,13975.000000
50%,3.000000e+00,2010-12-07 15:28:00,2.100000e+00,15255.000000
75%,1.000000e+01,2011-07-22 10:23:00,4.150000e+00,16797.000000
max,8.099500e+04,2011-12-09 12:50:00,3.897000e+04,18287.000000
std,1.727058e+02,NaN,1.235531e+02,1697.464450


## Duplicates

In [34]:
data.duplicated().sum()

np.int64(34335)

In [35]:
data.duplicated()

0          False
1          False
2          False
3          False
4          False
           ...  
1067366    False
1067367    False
1067368    False
1067369    False
1067370    False
Length: 1067371, dtype: bool

In [36]:
data = data.drop_duplicates(keep='first')
data.reset_index(drop=True)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
1033031,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
1033032,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
1033033,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
1033034,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [37]:
data.shape

(1033036, 8)

## NULL values

In [40]:
data.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [39]:
data = data.dropna(subset=['Customer ID'])

In [41]:
data['StockCode'].apply(type).value_counts()


StockCode
<class 'int'>    706085
<class 'str'>     91800
Name: count, dtype: int64

In [42]:
data['StockCode'] = data['StockCode'].astype(str)
data['StockCode'].apply(type).value_counts()

StockCode
<class 'str'>    797885
Name: count, dtype: int64

# Feature Engineering

In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 797885 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      797885 non-null  object        
 1   StockCode    797885 non-null  object        
 2   Description  797885 non-null  object        
 3   Quantity     797885 non-null  int64         
 4   InvoiceDate  797885 non-null  datetime64[ns]
 5   Price        797885 non-null  float64       
 6   Customer ID  797885 non-null  float64       
 7   Country      797885 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 54.8+ MB


In [44]:
data['TotalPrice'] = data['Quantity'] * data['Price']



In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 797885 entries, 0 to 1067370
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      797885 non-null  object        
 1   StockCode    797885 non-null  object        
 2   Description  797885 non-null  object        
 3   Quantity     797885 non-null  int64         
 4   InvoiceDate  797885 non-null  datetime64[ns]
 5   Price        797885 non-null  float64       
 6   Customer ID  797885 non-null  float64       
 7   Country      797885 non-null  object        
 8   TotalPrice   797885 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 60.9+ MB


In [ ]:
# RFM : 
# R : Recency when recently they bought 
# Frequency how frequently they  buy
# Monetary how much money are they spending it on



# RFM : 
### R : Recency when recently they bought Frequency how frequently they  buy Monetary how much money are they spending it on


In [46]:
import datetime as dt

# Define reference date (e.g., last invoice date + 1)
reference_date = data['InvoiceDate'].max() + dt.timedelta(days=1)

# Calculate recency (days since last purchase)
recency = data.groupby('Customer ID')['InvoiceDate'].max().reset_index()
recency['Recency'] = (reference_date - recency['InvoiceDate']).dt.days


In [49]:
frequency = data.groupby('Customer ID')['Invoice'].nunique().reset_index()
frequency.rename(columns={'Invoice': 'Frequency'}, inplace=True)


In [50]:
monetary = data.groupby('Customer ID')['TotalPrice'].sum().reset_index()
monetary.rename(columns={'TotalPrice': 'Monetary'}, inplace=True)


In [51]:
rfm = recency.merge(frequency, on='Customer ID')
rfm = rfm.merge(monetary, on='Customer ID')
rfm.head()

,Customer ID,InvoiceDate,Recency,Frequency,Monetary
0,12346.0,2011-01-18 10:17:00,326,17,-51.74
1,12347.0,2011-12-07 15:52:00,2,8,4921.53
2,12348.0,2011-09-25 13:13:00,75,5,2019.40
3,12349.0,2011-11-21 09:51:00,19,5,4404.54
4,12350.0,2011-02-02 16:01:00,310,1,334.40


In [52]:
rfm = rfm[rfm["Monetary"] > 0.0]
rfm['Monetary'].min()

np.float64(1.7763568394002505e-15)

In [53]:
rfm.head()

,Customer ID,InvoiceDate,Recency,Frequency,Monetary
1,12347.0,2011-12-07 15:52:00,2,8,4921.53
2,12348.0,2011-09-25 13:13:00,75,5,2019.40
3,12349.0,2011-11-21 09:51:00,19,5,4404.54
4,12350.0,2011-02-02 16:01:00,310,1,334.40
5,12351.0,2010-11-29 15:23:00,375,1,300.93


In [54]:
rfm.shape

(5843, 5)

In [55]:
# Recency: lower recency (more recent) = higher rank
rfm['R_rank'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1]).astype(int)

# Frequency: higher frequency = higher rank
rfm['F_rank'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)

# Monetary: higher spending = higher rank
rfm['M_rank'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5]).astype(int)


In [56]:
rfm.head()

,Customer ID,InvoiceDate,Recency,Frequency,Monetary,R_rank,F_rank,M_rank
1,12347.0,2011-12-07 15:52:00,2,8,4921.53,5,4,5
2,12348.0,2011-09-25 13:13:00,75,5,2019.40,3,3,4
3,12349.0,2011-11-21 09:51:00,19,5,4404.54,4,3,5
4,12350.0,2011-02-02 16:01:00,310,1,334.40,2,1,2
5,12351.0,2010-11-29 15:23:00,375,1,300.93,2,1,2


## StockCode Vs Country based on 